# @2
# GHI CHÚ
# I) File này dùng để mã hoá tên khách hàng trong bảng sổ chi tiết bán hàng. Sau đó  tạo và đưa dữ liệu vào bảng SO_CHI_TIET trong CSDL Ban_Hang
# II) Sử dụng dữ liệu sổ chi tiết đã qua xử lý bằng file preprocess và dữ liệu từ bảng KHACH_HANG trong CSDL Ban_Hang

In [2]:
import pandas as pd
import numpy as np
import pyodbc

In [3]:
SCT20 = pd.read_csv('D://THUCTAP//data_da_xuly//SCT_20.csv', encoding='utf-16', sep='\t')
SCT21 = pd.read_csv('D://THUCTAP//data_da_xuly//SCT_21.csv', encoding='utf-16', sep='\t')
SCT22 = pd.read_csv('D://THUCTAP//data_da_xuly//SCT_22.csv', encoding='utf-16', sep='\t')

In [4]:
SCT21.head(3)

,Ngày chứng từ,Số hóa đơn,Mã hàng,Tên hàng,ĐVT,Số lượng bán,Đơn giá,Doanh số bán,Đơn giá vốn,Giá vốn,Tên khách hàng
0,2021-01-01,'0000228,TL_TL,Thuốc lá Thăng Long B.C,Thùng,3,3755000.0,11265000,3740000.00,11220000,Hoàng Thị Hương
1,2021-01-01,'0000228,TL_TLSLIM,Thuốc lá Thăng Long Slim,Thùng,2,4215000.0,8430000,4195000.00,8390000,Hoàng Thị Hương
2,2021-01-02,'0000229,TL_BLA,Thuốc lá Blue Seal Apple Slim,Thùng,1,3770000.0,3770000,3739999.91,3740000,Đại Lý Tiến Thúy


# Nối mã KH

In [5]:
query = "SELECT * FROM KHACH_HANG"
KH = pd.read_sql_query(query, conn)

In [6]:
KH.head(3)

,Ten_khach_hang,Ma_KH
0,Vũ Như Hùng,KH00001
1,Hồ Thị Xuân,KH00002
2,Đại lý Tú Hạnh,KH00003


In [7]:
def merge_SCT(SCT,KH):
    SCT = SCT.merge(KH, how='left', left_on = 'Tên khách hàng', right_on = 'Ten_khach_hang')
    # Drop the redundant key column 'Ma_hoa_don'
    SCT.drop(columns=['Ten_khach_hang'], inplace=True)
    return SCT

In [8]:
SCT_20 = merge_SCT(SCT20,KH)
SCT_21 = merge_SCT(SCT21,KH)
SCT_22 = merge_SCT(SCT22,KH)

In [9]:
SCT_21

,Ngày chứng từ,Số hóa đơn,Mã hàng,Tên hàng,ĐVT,Số lượng bán,Đơn giá,Doanh số bán,Đơn giá vốn,Giá vốn,Tên khách hàng,Ma_KH
0,2021-01-01,'0000228,TL_TL,Thuốc lá Thăng Long B.C,Thùng,3,3755000.0,11265000,3740000.00,11220000,Hoàng Thị Hương,KH00116
1,2021-01-01,'0000228,TL_TLSLIM,Thuốc lá Thăng Long Slim,Thùng,2,4215000.0,8430000,4195000.00,8390000,Hoàng Thị Hương,KH00116
2,2021-01-02,'0000229,TL_BLA,Thuốc lá Blue Seal Apple Slim,Thùng,1,3770000.0,3770000,3739999.91,3740000,Đại Lý Tiến Thúy,KH00005
3,2021-01-02,'0000229,TL_RC,Thuốc lá Thăng Long Round Corner,Thùng,1,5750000.0,5750000,5735454.27,5735454,Đại Lý Tiến Thúy,KH00005
4,2021-01-02,'0000229,TL_SP,Thuốc lá Sapa B.M,Thùng,2,2265000.0,4530000,2244990.71,4489981,Đại Lý Tiến Thúy,KH00005
...,...,...,...,...,...,...,...,...,...,...,...,...
6640,2021-12-31,'0001100,TL_SP,Thuốc lá Sapa B.M,Thùng,2,2520000.0,5040000,2496594.68,4993189,Hà Toàn,KH00126
6641,2021-12-31,'0001100,TL_TL,Thuốc lá Thăng Long B.C,Thùng,65,3910000.0,254150000,3912458.78,254309821,Hà Toàn,KH00126
6642,2021-12-31,'0001100,TL_TLSLIM,Thuốc lá Thăng Long Slim,Thùng,15,4215000.0,63225000,4195000.00,62925000,Hà Toàn,KH00126
6643,2021-12-31,'0001100,TL_VINADS,Thuốc lá Vinataba Demi Slims,Kiện,1,3470000.0,3470000,3457685.98,3457686,Hà Toàn,KH00126


In [10]:
def Format_SCT(df):
    df['Ma_KH'] = df['Ma_KH'].fillna('null')
    columns_to_drop = ['Diễn giải chung','ĐVT','Tên hàng','Tên khách hàng']
    # Check if columns exist before attempting to drop them
    columns_to_drop_existing = [col for col in columns_to_drop if col in df.columns]
    if columns_to_drop_existing:
        df.drop(columns=columns_to_drop_existing, inplace=True)    
    return df    

In [11]:
SCT_20=Format_SCT(SCT_20)
SCT_21=Format_SCT(SCT_21)
SCT_22=Format_SCT(SCT_22)

# B2 ĐƯA DỮ LIỆU VÀO CSDL

In [3]:
# Kết nối với CSDL
conn = pyodbc.connect(
Trusted_Connection = "Yes",
Driver = '{ODBC Driver 17 for SQL Server}',
Server = "DESKTOP-MDDIIDJ\MSSQLSERVER01",
Database = 'Ban_Hang')
cursor = conn.cursor()

In [12]:
#Tạo bảng 

cursor.execute("CREATE TABLE SO_CHI_TIET(Ngay_chung_tu date, So_hoa_don varchar(10), Ma_hang varchar(20), SL_ban int, Don_gia float, Doanh_so float, Don_gia_von float, Gia_von float, Ma_KH varchar(8) DEFAULT '?')")

In [13]:
def Insert_SCT(df):
    for row in df.itertuples(index=False):
        so_hoa_don = row[1][1:]
        cursor.execute('''
                        INSERT INTO BAN_HANG.dbo.SO_CHI_TIET(Ngay_chung_tu, So_hoa_don, Ma_hang, SL_ban, Don_gia, Doanh_so, Don_gia_von, Gia_von, Ma_KH )
                        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''',
                       row[0], so_hoa_don, row[2], row[3], row[4], row[5], row[6], row[7], row[8]
                      )
    conn.commit()

In [14]:
#thêm 2020
Insert_SCT(SCT_20)

In [15]:
#thêm 2021
Insert_SCT(SCT_21)

In [16]:
#lấy ra mã KH có trong CSDL để thực hiện nối
#query = "SELECT * FROM KHACH_HANG"
#KH = pd.read_sql_query(query, conn)

In [17]:
#SCT_22 = merge_SCT(SCT22,KH)
#SCT_22=Format_SCT(SCT_22)
##thêm 2022
Insert_SCT(SCT_22)